# Create Figures for Group AGF Paper

For each group, we should have 4 figures that make up a panel. We can organize these figures in Adobe Illustrator for the final figures.
- Loss plot with clear drops
- Model output power over epoch
- Weights over epoch (ideally one weight per irreps, but for MNIST just choose ~5.)
- Output prediction vs target over epoch

Action Plan:
- Load Model
- recreate power and loss figures

Weights figure
- go through weights and label them with their predomonant frequency
- Choose 1 weight per frequency, and plot these over time. In AI outline these weights with the same color as their power
- in the loss plot, try to make the color of the line (starting at the drop) match the color of the power and the irreps

Output figure
- detect jumps in power
- right after each jump, save epoch value for label
- at that epoch, plot Y[i] and model[X[i]] output.

In [ ]:
import numpy as np
import random
import torch
import os
import torch.nn as nn
import torch.optim as optim
import shutil
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator

import importlib
import pickle

import group_agf.binary_action_learning.models as models
import group_agf.binary_action_learning.datasets as datasets
import group_agf.binary_action_learning.power as power
import group_agf.binary_action_learning.plot as plot

from escnn.group import *

In [ ]:
# parameters
config = {
    "model_save_dir": "/tmp/adele/",
    "dataset_fraction": 0.3,  # fraction of the total dataset to train on
    "group_name": 'dihedral',  # 'dihedral', 'cnxcn', 'octahedral'
    "mnist_digit": 4,
    "group_n": 4,  # n in Dn [3, 4, 5]
    "image_length": 10,  # length of one side of the square image patch
    # Learning Parameters
    "seed": 10,
    "init_scale": 1e-2,
    "lr": 0.001,
    "mom": 0.9,
    # Training parameters
    "epochs": 5000,
    "verbose_interval": 100,
    "batch_size": 128,
    "frequencies_to_learn": 3,
    "run_start_time": "10-31_14-42-46",
}

config["group"] = DihedralGroup(config["group_n"])
config["group_size"] = config["group"].order()

print(f"Group name: {config['group_name']}, group size: {config['group_size']}")

In [ ]:
from pyexpat import model


model_save_path = models.get_model_save_path(config)
print(model_save_path)
# model_save_path = '/tmp/adele/model_group_namedihedral_group_size8_frac0.3_init0.01_lr0.01_mom0.9_bs128_epochs5000_seed10_run_start10-31_14-25-32.pkl'
# print(model_save_path)

# Load training history and model parameters from the specified path
with open(model_save_path, 'rb') as f:
    training_history = pickle.load(f)

loss_history = training_history["loss_history"]
accuracy_history = training_history["accuracy_history"]
param_history = training_history["param_history"]

In [ ]:
X, Y, template = datasets.load_dataset(config)
template_power = power.GroupPower(template, config['group'])
X, Y, device = datasets.move_dataset_to_device_and_flatten(X, Y, device=None)

In [ ]:
# Create the model instance (matching the one that was trained)
model = models.TwoLayerNet(
    group_size=config['group_size'],
    hidden_size=None,  # uses default if not provided
    nonlinearity='square',  # adjust if needed according to training
    init_scale=config['init_scale'],
    output_scale=1.0
).to(device)

# Restore the model parameters from the last saved point in param_history
final_params = param_history[-1]
with torch.no_grad():
    model.U.copy_(final_params['U'])
    model.V.copy_(final_params['V'])
    model.W.copy_(final_params['W'])

In [ ]:
power_over_training_plot, freq_colors = plot.plot_training_power_over_time(
    template_power, 
    model, 
    device, 
    param_history, 
    X, 
    config['group_name'], 
    save_path=None, 
    show=False,
    return_freq_colors=True
)

In [ ]:
loss_plot = plot.plot_loss_curve(loss_history, template_power, show=False, freq_colors=freq_colors)


In [ ]:
irreps_plot = plot.plot_irreps(config['group'], config['group_name'], show=False)


In [ ]:
neuron_indices = list(range(config['group_size'] * config['frequencies_to_learn']))
neuron_weights_plot = plot.plot_neuron_weights(
    config['group_name'],
    config['group'],
    model,
    config['group_size'],
    neuron_indices=neuron_indices,
    show=True
)

In [ ]:
model_predictions_plot = plot.plot_model_outputs(config['group_name'], config['group_size'], model, X, Y, idx=13, step=1, show=True)  
